# Sensor Utils


In [1]:
import os

# kalasiris requires ISISROOT and ISISDATA environment variables to be set.
os.environ['ISISROOT'] = '/usr/local/Caskroom/miniconda/base/envs/isis-prod'
os.environ['ISISDATA'] = '/Volumes/usgs-data/isis-data'
import kalasiris as isis
import subprocess
import urllib.request

from csmapi import csmapi
from knoten import csm, sensor_utils

from knoten.shape import Ellipsoid
from knoten.illuminator import Illuminator

import ale
import json

In [5]:
# File Paths

cissurl = [
    'https://planetarydata.jpl.nasa.gov/img/data/cassini/cassini_orbiter/coiss_2038/data/1573076626_1573088441/N1573082850_1.IMG',
    'https://planetarydata.jpl.nasa.gov/img/data/cassini/cassini_orbiter/coiss_2038/data/1573076626_1573088441/N1573082850_1.LBL'
]

image_dir = 'data'
# cissfile [0] is the .IMG and [1] is the .LBL
cissfile = [ os.path.join(image_dir, os.path.basename(url)) for url in cissurl]

In [6]:
# Download Files

downloader = urllib.request.URLopener()

for url, file in zip(cissurl, cissfile):
    if not os.path.isfile(file):
        downloader.retrieve(url, file)

In [7]:
cub_loc = os.path.splitext(cissfile[0])[0] + '.cub'

try: 
    isis.ciss2isis(from_=cissfile[1], to=cub_loc)
except subprocess.CalledProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc)
except subprocess.CalledProcessError as e:
    print(e.stderr)

## Create a usgscsm sensor model

In [8]:
fileName = "data/N1573082850_1.cub"

kernels = ale.util.generate_kernels_from_cube(fileName, expand=True)
isd_string = ale.loads(fileName, props={'kernels': kernels})
csm_isd = os.path.splitext(fileName)[0] + '.json'

with open(csm_isd, 'w') as isd_file:
    isd_file.write(isd_string)

/usr/local/Caskroom/miniconda/base/envs/knoten/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


## Run Sensor Utils with usgscsm sensor model and image point

In [9]:
camera = csm.create_csm(csm_isd)
image_pt = csmapi.ImageCoord(511.5, 511.5)
shape = Ellipsoid.from_csm_sensor(camera)
illuminator = Illuminator()

In [10]:
phaseAngle = sensor_utils.phase_angle(image_pt, camera, shape, illuminator)

phaseAngle

np.float64(38.87212509629895)

In [11]:
emissionAngle = sensor_utils.emission_angle(image_pt, camera, shape)

emissionAngle

np.float64(49.60309924893989)

In [12]:
slantDistance = sensor_utils.slant_distance(image_pt, camera, shape)

slantDistance

np.float64(2903512972.146115)

In [13]:
targetCenterDistance = sensor_utils.target_center_distance(image_pt, camera)

targetCenterDistance

np.float64(2943536048.858226)

In [14]:
subSpacecraftPoint = sensor_utils.sub_spacecraft_point(image_pt, camera)

subSpacecraftPoint

LatLon(lat=np.float64(3.2229625890973583), lon=np.float64(258.6197326526089))

In [15]:
localRadius = sensor_utils.local_radius(image_pt, camera, shape)

localRadius

np.float64(59096282.024265066)

In [16]:
rightAscDec = sensor_utils.right_ascension_declination(image_pt, camera)

rightAscDec

(np.float64(79.34815579474038), np.float64(-2.7790780986459485))

In [17]:
lineResolution = sensor_utils.line_resolution(image_pt, camera, shape)

lineResolution

np.float64(17397.96094194587)

In [18]:
sampleResolution = sensor_utils.sample_resolution(image_pt, camera, shape)

sampleResolution

np.float64(17397.93370038153)

In [19]:
pixelResolution = sensor_utils.pixel_resolution(image_pt, camera, shape)

pixelResolution

np.float64(17397.9473211637)